In [ ]:
# !pip install transformers==4.25.1
# !pip install pytorch-lightning==1.6.4
# !pip install timm==0.5.4
# !pip install gradio
# !pip install donut-python

In [1]:
import argparse
import gradio as gr
import torch
from PIL import Image
from transformers import pipeline

/Users/boyander/KIRBIC/content-talleres-jornadas/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
def demo_process_vqa(input_img, question):
    global pipeline, task_prompt, task_name
    input_img = Image.fromarray(input_img)
    user_prompt = task_prompt.replace("{user_input}", question)
    output = pipeline(input_img, prompt=user_prompt)["predictions"][0]
    return output



def demo_process(input_img):
    global pipeline, task_prompt, task_name
    input_img = Image.fromarray(input_img)
    output = pipeline(image=input_img, prompt=task_prompt)["predictions"][0]
    return output

In [4]:

parser = argparse.ArgumentParser()
parser.add_argument("--task", type=str, default="cord-v2")
parser.add_argument("--pretrained_path", type=str, default="naver-clova-ix/donut-base-finetuned-cord-v2")
args, left_argv = parser.parse_known_args()

task_name = args.task
if "docvqa" == task_name:
    task_prompt = "<s_docvqa><s_question>{user_input}</s_question><s_answer>"
else:  # rvlcdip, cord, ...
    task_prompt = f"<s_{task_name}>"

pipeline = pipeline(
    task="document-question-answering",
    model=args.pretrained_path,
    torch_dtype=torch.float16
)

#pretrained_model = DonutModel.from_pretrained(args.pretrained_path,ignore_mismatched_sizes=True)

# if torch.cuda.is_available():
#     pretrained_model.half()
#     device = torch.device("cuda")
#     pretrained_model.to(device)
# else:
#     pass
#     #pretrained_model.encoder.to(torch.bfloat16)

# pretrained_model.eval()


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use mps:0


In [ ]:
image=Image.open("../../ticket.jpg").resize([512,512])
pipeline(image,question="What time is the coffee break?")

AttributeError: 'Image' object has no attribute 'get'

In [ ]:
demo = gr.Interface(
    fn=demo_process_vqa if task_name == "docvqa" else demo_process,
    inputs=["image", "text"] if task_name == "docvqa" else "image",
    outputs="json",
    title=f"Donut 🍩 demonstration for `{task_name}` task",
)
demo.launch(debug=True)